# Setup

In [ ]:
# The examples in this notebook use a set of nine benchmarks described in our publication.
# These benchmarks can be downloaded from: https://github.com/nadavbra/proteinbert_data_files/tree/master/protein_benchmarks
# Download the benchmarks into a directory on your machine and set the following variable to the path of that directory.
BENCHMARKS_DIR = '/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks'


In [ ]:
import csv
# To convert text file into labelled csv
with open("/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks/Custom_inputs/input_hem.txt") as f:
    l = f.readlines()
    l = [i.strip() for i in l]
    with open("/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks/Custom_inputs/input_hem.csv","w") as f2:
        writer = csv.writer(f2)
        heading = ["sequence","label"]
        writer.writerow(heading)
        for i in range(100):
            to_write = []
            to_write.append(l[i])
            to_write.append(1)
            writer.writerow(to_write)
        for i in range(100,200):
            to_write = []
            to_write.append(l[i])
            to_write.append(0)
            writer.writerow(to_write)

# Fine-tune the model for the signal peptide benchmark

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from IPython.display import display
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

# Set random seeds for reproducibility
seed = 100
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Benchmark name and output specification
# BENCHMARK_NAME = 'scop'
# 

OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0,1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

# Directory containing benchmarks

# Loading the dataset
train_set_file_path = "/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks/Custom_inputs/hem.csv"
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
train_set, valid_set = train_test_split(train_set, stratify=train_set['label'], test_size=0.1, random_state=100)

test_set_file_path = "/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks/Custom_inputs/input_hem.csv"
test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()

print(f'{len(train_set)} training set records, {len(valid_set)} validation set records, {len(test_set)} test set records.')
print(valid_set.head(10))

# Loading the pre-trained model and fine-tuning it on the loaded dataset
batch_size = 45
seq_len = 200
final = seq_len * 2
pretrained_model_generator, input_encoder = load_pretrained_model()

# get_model_with_hidden_layers_as_outputs gives the model output access to the hidden layers (on top of the output)
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function=get_model_with_hidden_layers_as_outputs, dropout_rate=0.4849098099486382)

training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience=1, factor=0.25, min_lr=8.741510119145999e-05, verbose=1),
    keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
]

finetune(model_generator, input_encoder, OUTPUT_SPEC, train_set['sequence'], train_set['label'], valid_set['sequence'], valid_set['label'],
         seq_len=seq_len, batch_size=batch_size, max_epochs_per_stage=20
         , lr=8.741510119145999e-05, begin_with_frozen_pretrained_layers=True,
         lr_with_frozen_pretrained_layers=1e-02, n_final_epochs=1, final_seq_len=final, final_lr=1e-05, callbacks=training_callbacks)

    # Evaluating the performance on the test set
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_set['sequence'], test_set['label'],
                                            start_seq_len=seq_len, start_batch_size=batch_size)
print("seq_len = %d" % seq_len)
print('Test-set performance:')
display(results)

print('Confusion matrix:')
display(confusion_matrix)

# Finding number of uniquely labeled sequences

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

BENCHMARKS_DIR = '/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks'
BENCHMARK_NAME = 'ProFET_NP_SP_Cleaved'
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0,1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

train_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.train.csv' % BENCHMARK_NAME)
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
#train_set, valid_set = train_test_split(train_set, stratify=train_set['label'], test_size=0.1, random_state=100)

test_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.test.csv' % BENCHMARK_NAME)
test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()


print("Training Set pos : ", len(train_set[train_set["label"] == 1]))
print("Training Set neg : ", len(train_set[train_set["label"] == 0]))
print(len(train_set))
print("Testing Set pos : ", len(test_set[test_set["label"] == 1]))
print("Testing Set neg : ", len(test_set[test_set["label"] == 0]))
print(len(test_set))

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from IPython.display import display
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

BENCHMARKS_DIR = '/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks'
BENCHMARK_NAME = 'scop'
OUTPUT_TYPE = OutputType(False, 'categorical')
UNIQUE_LABELS = ['a','b','c','d','e','f','g']
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

train_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.train.csv' % BENCHMARK_NAME)
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
train_set, valid_set = train_test_split(train_set, stratify=train_set['label'], test_size=0.1, random_state=100)

test_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.test.csv' % BENCHMARK_NAME)
test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()


print("Training Set a : ", len(train_set[train_set["label"] == 'a']))
print("Training Set b : ", len(train_set[train_set["label"] == 'b']))
print("Training Set c : ", len(train_set[train_set["label"] == 'c']))
print("Training Set d : ", len(train_set[train_set["label"] == 'd']))
print("Training Set e : ", len(train_set[train_set["label"] == 'e']))
print("Training Set f : ", len(train_set[train_set["label"] == 'f']))
print("Training Set g : ", len(train_set[train_set["label"] == 'g']))
print(len(train_set))
print("Valid Set a : ", len(valid_set[valid_set["label"] == 'a']))
print("Valid Set b  : ", len(valid_set[valid_set["label"] == 'b']))
print("Valid Set c : ", len(valid_set[valid_set["label"] == 'c']))
print("Valid Set d : ", len(valid_set[valid_set["label"] == 'd']))
print("Valid Set e : ", len(valid_set[valid_set["label"] == 'e']))
print("Valid Set f : ", len(valid_set[valid_set["label"] == 'f']))
print("Valid Set g : ", len(valid_set[valid_set["label"] == 'g']))
print(len(valid_set))
print("Testing Set a : ", len(test_set[test_set["label"] == 'a']))
print("Testing Set b : ", len(test_set[test_set["label"] == 'b']))
print("Testing Set c : ", len(test_set[test_set["label"] == 'c']))
print("Testing Set d : ", len(test_set[test_set["label"] == 'd']))
print("Testing Set e : ", len(test_set[test_set["label"] == 'e']))
print("Testing Set f : ", len(test_set[test_set["label"] == 'f']))
print("Testing Set g : ", len(test_set[test_set["label"] == 'g']))
print(len(test_set))

# Finding Embeddings

In [ ]:
from proteinbert import model_generation as mg
import pandas as pd
import csv
train_set_file_path = "/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks/ProFET_NP_SP_Cleaved.train.csv"
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
f = open("/Users/yeyatiprasher/Coding/Internship/protein_bert/NPCleaved_embed.csv","w")
writer = csv.writer(f)
for i in train_set['seq']:
    embed = mg.tokenize_seqs(i,512)
    for j in embed:
        writer.writerow(j)

In [ ]:
from proteinbert import model_generation as mg
import pandas as pd
import csv
train_set_file_path = "/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks/scop.train.csv"
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
f = open("/Users/yeyatiprasher/Coding/Internship/protein_bert/scop_embed.csv","w")
print(len(train_set))
writer = csv.writer(f)
for i in train_set['seq']:
    embed = mg.tokenize_seqs(i,512)
    for j in embed:
        writer.writerow(j)

# Run all benchmarks

In [ ]:
import os

import pandas as pd
from IPython.display import display

from tensorflow import keras

from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len, log
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

BENCHMARKS = [
    # name, output_type
    ('signalP_binary', OutputType(False, 'binary')),
    ('fluorescence', OutputType(False, 'numeric')),
    ('remote_homology', OutputType(False, 'categorical')),
    ('stability', OutputType(False, 'numeric')),
    ('scop', OutputType(False, 'categorical')),
    ('secondary_structure', OutputType(True, 'categorical')),
    ('disorder_secondary_structure', OutputType(True, 'binary')),
    ('ProFET_NP_SP_Cleaved', OutputType(False, 'binary')),
    ('PhosphositePTM', OutputType(True, 'binary')),
]

settings = {
    'max_dataset_size': None,
    'max_epochs_per_stage': 5,
    'seq_len': 512,
    'batch_size': 32,
    'final_epoch_seq_len': 1024,
    'initial_lr_with_frozen_pretrained_layers': 1e-02,
    'initial_lr_with_all_layers': 1e-04,
    'final_epoch_lr': 1e-05,
    'dropout_rate': 0.5,
    'training_callbacks': [
        keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-05, verbose = 1),
        keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True),
    ],
}

####### Uncomment for debug mode
# settings['max_dataset_size'] = 500
# settings['max_epochs_per_stage'] = 1

def run_benchmark(benchmark_name, pretraining_model_generator, input_encoder, pretraining_model_manipulation_function = None):
    
    log('========== %s ==========' % benchmark_name)  
    
    output_type = get_benchmark_output_type(benchmark_name)
    log('Output type: %s' % output_type)
    
    train_set, valid_set, test_set = load_benchmark_dataset(benchmark_name)        
    log(f'{len(train_set)} training set records, {len(valid_set)} validation set records, {len(test_set)} test set records.')
    
    if settings['max_dataset_size'] is not None:
        log('Limiting the training, validation and test sets to %d records each.' % settings['max_dataset_size'])
        train_set = train_set.sample(min(settings['max_dataset_size'], len(train_set)), random_state = 0)
        valid_set = valid_set.sample(min(settings['max_dataset_size'], len(valid_set)), random_state = 0)
        test_set = test_set.sample(min(settings['max_dataset_size'], len(test_set)), random_state = 0)
    
    if output_type.is_seq or output_type.is_categorical:
        train_set['label'] = train_set['label'].astype(str)
        valid_set['label'] = valid_set['label'].astype(str)
        test_set['label'] = test_set['label'].astype(str)
    else:
        train_set['label'] = train_set['label'].astype(float)
        valid_set['label'] = valid_set['label'].astype(float)
        test_set['label'] = test_set['label'].astype(float)
        
    if output_type.is_categorical:
        
        if output_type.is_seq:
            unique_labels = sorted(set.union(*train_set['label'].apply(set)) | set.union(*valid_set['label'].apply(set)) | \
                    set.union(*test_set['label'].apply(set)))
        else:
            unique_labels = sorted(set(train_set['label'].unique()) | set(valid_set['label'].unique()) | set(test_set['label'].unique()))
            
        log('%d unique lebels.' % len(unique_labels))
    elif output_type.is_binary:
        unique_labels = [0, 1]
    else:
        unique_labels = None
        
    output_spec = OutputSpec(output_type, unique_labels)
    model_generator = FinetuningModelGenerator(pretraining_model_generator, output_spec, pretraining_model_manipulation_function = \
            pretraining_model_manipulation_function, dropout_rate = settings['dropout_rate'])
    finetune(model_generator, input_encoder, output_spec, train_set['seq'], train_set['label'], valid_set['seq'], valid_set['label'], \
            seq_len = settings['seq_len'], batch_size = settings['batch_size'], max_epochs_per_stage = settings['max_epochs_per_stage'], \
            lr = settings['initial_lr_with_all_layers'], begin_with_frozen_pretrained_layers = True, lr_with_frozen_pretrained_layers = \
            settings['initial_lr_with_frozen_pretrained_layers'], n_final_epochs = 1, final_seq_len = settings['final_epoch_seq_len'], \
            final_lr = settings['final_epoch_lr'], callbacks = settings['training_callbacks'])
    
    for dataset_name, dataset in [('Training-set', train_set), ('Validation-set', valid_set), ('Test-set', test_set)]:
        
        log('*** %s performance: ***' % dataset_name)
        results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, output_spec, dataset['seq'], dataset['label'], \
                start_seq_len = settings['seq_len'], start_batch_size = settings['batch_size'])
    
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(results)
        
        if confusion_matrix is not None:
            with pd.option_context('display.max_rows', 16, 'display.max_columns', 10):
                log('Confusion matrix:')
                display(confusion_matrix)
                
    return model_generator

def load_benchmark_dataset(benchmark_name):
    
    train_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.train.csv' % benchmark_name)
    valid_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.valid.csv' % benchmark_name)
    test_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.test.csv' % benchmark_name)
    
    train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
    test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()
          
    if os.path.exists(valid_set_file_path):
        valid_set = pd.read_csv(valid_set_file_path).dropna().drop_duplicates()
    else:
        log(f'Validation set {valid_set_file_path} missing. Splitting training set instead.')
        train_set, valid_set = train_test_split(train_set, stratify = train_set['label'], test_size = 0.1, random_state = 0)
    
    return train_set, valid_set, test_set

def get_benchmark_output_type(benchmark_name):
    for name, output_type in BENCHMARKS:
        if name == benchmark_name:
            return output_type
        
pretrained_model_generator, input_encoder = load_pretrained_model()

for benchmark_name, _ in BENCHMARKS:
    run_benchmark(benchmark_name, pretrained_model_generator, input_encoder, pretraining_model_manipulation_function = \
            get_model_with_hidden_layers_as_outputs)
        
log('Done.')

# Visualizing the attention layers

You can run this only after you have fine-tuned the model on a benchmark (e.g. signal peptide) and obtained *model_generator*.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

BENCHMARK_DISPLAY_NAME = 'Signal peptide'
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)
TEST_SET_FILE_PATH = '/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks/signalP_binary.train.csv'
IDEAL_LEN = 80

def calculate_attentions(model, input_encoder, seq, seq_len = None):
    
    from tensorflow.keras import backend as K
    from proteinbert.tokenization import index_to_token
    
    if seq_len is None:
        seq_len = len(seq) + 2
    
    X = input_encoder.encode_X([seq], seq_len)
    (X_seq,), _ = X
    seq_tokens = list(map(index_to_token.get, X_seq))

    model_inputs = [layer.input for layer in model.layers if 'InputLayer' in str(type(layer))][::-1]
    model_attentions = [layer.calculate_attention(layer.input) for layer in model.layers if 'GlobalAttention' in str(type(layer))]
    invoke_model_attentions = K.function(model_inputs, model_attentions)
    attention_values = invoke_model_attentions(X)
    
    attention_labels = []
    merged_attention_values = []

    for attention_layer_index, attention_layer_values in enumerate(attention_values):
        for head_index, head_values in enumerate(attention_layer_values):
            attention_labels.append('Attention %d - head %d' % (attention_layer_index + 1, head_index + 1))
            merged_attention_values.append(head_values)

    attention_values = np.array(merged_attention_values)
    
    return attention_values, seq_tokens, attention_labels

def plot_attention(attention_values, seq_tokens, attention_labels, ax, cmap = 'Reds', vmin = 0, vmax = None, text_value_threshold = 0.1):

    heatmap = ax.pcolor(attention_values.transpose(), cmap = cmap, vmin = vmin, vmax = vmax)

    ax.set_xticks(np.arange(len(attention_labels)) + 0.5)
    ax.set_xticklabels(attention_labels, rotation = 45, ha = 'right', fontsize = 12)
    ax.set_yticks(np.arange(len(seq_tokens)) + 0.5)
    ax.set_yticklabels(seq_tokens, fontsize = 12)

    for i, row in enumerate(attention_values):
        for j, value in enumerate(row):
            if abs(value) >= text_value_threshold:
                add_plus_sign = attention_values.min() < 0 and value > 0
                plus_sign = '+' if add_plus_sign else ''
                ax.text(i + 0.5, j + 0.5, plus_sign + '%d%%' % (100 * value), color = 'white', ha = 'center', va = 'center', \
                        fontsize = 9, fontweight = 'bold', fontstretch = 'condensed')
                
test_set = pd.read_csv(TEST_SET_FILE_PATH)
chosen_index = ((test_set['seq'].str.len() - IDEAL_LEN).abs()).sort_values().index[0]
seq = test_set.loc[chosen_index, 'seq']
label = test_set.loc[chosen_index, 'label']
                
seq_len = len(seq) + 2

pretrained_model_generator, input_encoder = load_pretrained_model()
model = pretrained_model_generator.create_model(seq_len)
pretrained_attention_values, pretrained_seq_tokens, pretrained_attention_labels = calculate_attentions(model, input_encoder, seq, \
        seq_len = seq_len)

model = model_generator.create_model(seq_len)
finetuned_attention_values, finetuned_seq_tokens, finetuned_attention_labels = calculate_attentions(model, input_encoder, seq, \
        seq_len = seq_len)
assert finetuned_seq_tokens == pretrained_seq_tokens
assert finetuned_attention_labels == pretrained_attention_labels[:len(finetuned_attention_labels)]

fig, axes = plt.subplots(ncols = 4, figsize = (20, 0.2 * seq_len), gridspec_kw = dict(width_ratios = [1, 5, 1, 5]))
fig.subplots_adjust(wspace = 0.3)

axes[0].barh(np.arange(seq_len), 100 * pretrained_attention_values.sum(axis = 0), color = '#EC7063')
axes[0].set_ylim((-0.5, seq_len - 0.5))
axes[0].set_yticks([])
axes[0].invert_xaxis()
axes[0].set_xlabel('Total atten. %', fontsize = 14)

vmax = pretrained_attention_values.max()
plot_attention(pretrained_attention_values, pretrained_seq_tokens, pretrained_attention_labels, axes[1], cmap = 'Reds', vmax = vmax, \
        text_value_threshold = 0.05)
axes[1].set_title('Only pre-training', fontsize = 16)

axes[2].barh(np.arange(seq_len), 100 * (finetuned_attention_values - pretrained_attention_values).sum(axis = 0), color = '#28B463')
axes[2].set_ylim((-0.5, seq_len - 0.5))
axes[2].set_yticks([])
axes[2].invert_xaxis()
axes[2].set_xlabel('Total atten. % diff', fontsize = 14)

attention_diff = finetuned_attention_values - pretrained_attention_values[:len(finetuned_attention_labels), :]
vmax = np.abs(attention_diff).max()
plot_attention(attention_diff, finetuned_seq_tokens, finetuned_attention_labels, axes[3], cmap = 'PiYG', vmin = -vmax, vmax = vmax, \
        text_value_threshold = 0.03)
axes[3].set_title('%s fine-tuning' % BENCHMARK_DISPLAY_NAME, fontsize = 16)

print(seq, label)

# Optimizing the model

In [ ]:
import os
import pandas as pd
import random
from IPython.display import display
import numpy as np
import tensorflow as tf

import optuna
from tensorflow import keras
from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

BENCHMARK_NAME = 'ProFET_NP_SP_Cleaved'
BENCHMARKS_DIR = '/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks'

# A local (non-global) binary output
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)
seed = 100
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
# Loading the dataset
train_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.train.csv' % BENCHMARK_NAME)
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
train_set, valid_set = train_test_split(train_set, stratify=train_set['label'], test_size=0.1, random_state=seed)




# Define the Optuna objective function
def objective(trial):
    # Suggest hyperparameters
    #batch_size = trial.suggest_int('batch_size', 20, 64)
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    #dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)

    test_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.test.csv' % BENCHMARK_NAME)
    test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()
    print('%d training set records, %d validation set records, %d test set records.' % (len(train_set), len(valid_set), len(test_set)))

    pretrained_model_generator, input_encoder = load_pretrained_model()
    model_generator = FinetuningModelGenerator(
        pretrained_model_generator, OUTPUT_SPEC, 
        pretraining_model_manipulation_function=get_model_with_hidden_layers_as_outputs,
        dropout_rate=0.4849098099486382
    )

    training_callbacks = [
        keras.callbacks.ReduceLROnPlateau(patience=1, factor=0.25, min_lr=1e-05, verbose=1),
        keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
    ]

    finetune(model_generator, input_encoder, OUTPUT_SPEC, train_set['seq'], train_set['label'], valid_set['seq'], valid_set['label'], 
             seq_len=512, batch_size=45, max_epochs_per_stage=1, lr=lr,
             begin_with_frozen_pretrained_layers=True, lr_with_frozen_pretrained_layers=1e-02, 
             n_final_epochs=1, final_seq_len=1024, final_lr=1e-05, callbacks=training_callbacks)

    results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_set['seq'], test_set['label'], 
                                                start_seq_len=512, start_batch_size=45)
    
    overall_accuracy = results.loc['All', 'AUC']
    print(overall_accuracy)
    return overall_accuracy

# Create an Optuna study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3)

# Print the best hyperparameters
print('Best hyperparameters: ', study.best_params)
print('Best accuracy: ', study.best_value)